# Preprocesamiento

In [5]:
import os
from nltk.stem import SnowballStemmer 
import nltk
import string
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
#Path de los datos 
#data_path = r'C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\training_txt'
#data_path = r'D:\U\7. Septimo\RI\ProyectoRI\data\training_txt'
data_path = r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\training_txt'

In [7]:
documents = [] #  Vector de documentos
for filename in os.listdir(data_path):
    if filename.endswith('.txt'): 
        path = os.path.join(data_path, filename) #Abrimos cada archivo 
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read() # leemos
            documents.append((filename, content))

In [8]:
#Path de las stopwords
#stopwords_path = r"C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\stopwords.txt"
#stopwords_path = r"D:\U\7. Septimo\RI\ProyectoRI\data\stopwords.txt"
stopwords_path = r"C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\stopwords.txt"

In [9]:
#Abrimos el archivo 
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines()) # leemos las stopwords

In [10]:
#Stemmer se usa para reducir las palabras a su raíz
stemmer = SnowballStemmer('english')

In [11]:
#Definimos una función que normaliza el texto con todos los requisitos necesarios:
def preprocess_text(text):
    text = text.lower() #conviertimos en minúsculas
    text = re.sub(r'\d+', '', text)  
    text = text.translate(str.maketrans('', '', string.punctuation))#eliminamos los signos de puntuación
    tokens = nltk.word_tokenize(text)#tokenizamos
    processed_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words] #aplicamos stemming
    return ' '.join(processed_tokens)

In [12]:
preprocessed_documents = [(filename, preprocess_text(content)) for filename, content in documents]

# Vectorizacion

In [13]:
# Extraer solo los contenidos preprocesados para la vectorización
preprocessed_contents = [content for _, content in preprocessed_documents]

### BoW 

In [14]:
# Inicializar el vectorizador
vectorizer_bow = CountVectorizer()
# Vectorizar los contenidos preprocesados
X_bow = vectorizer_bow.fit_transform(preprocessed_contents)
feature_names_bow = vectorizer_bow.get_feature_names_out()

# Motor de busqueda

### BoW

In [15]:
# Función auxiliar para convertir vectores a binarios
def binary_vector(vector):
    return (vector > 0).astype(int)

In [16]:
# Función auxiliar para calcular la similitud de Jaccard
def jaccard_similarity(bin_vec1, bin_vec2):
    intersection = np.sum(np.minimum(bin_vec1, bin_vec2))
    union = np.sum(np.maximum(bin_vec1, bin_vec2))
    return intersection / union if union != 0 else 0

In [17]:
# Celda 2: Definir la clase del motor de búsqueda
class SearchEngine:
    def __init__(self, vectorizer, document_vectors, documents, indices):
        self.vectorizer = vectorizer
        self.document_vectors = document_vectors
        self.documents = documents
        self.indices = indices

    def search(self, query):
        query_vector = self.vectorizer.transform([query]).toarray()
        bin_query_vector = binary_vector(query_vector)

        scores = []
        for i, doc_vector in enumerate(self.document_vectors):
            bin_doc_vector = binary_vector(doc_vector.toarray())
            similarity = jaccard_similarity(bin_query_vector, bin_doc_vector)
            scores.append((similarity, self.indices[i]))
        scores.sort(reverse=True, key=lambda x: x[0])
        return scores

In [18]:
# Crear instancia del motor de búsqueda
documents = [doc for doc in preprocessed_contents]
indices = [index for index, _ in preprocessed_documents]
engine = SearchEngine(vectorizer_bow, X_bow, documents, indices)


In [19]:
def leer_documentos_relevantes(archivo):
    documentos_relevantes = {}
    with open(archivo, 'r') as file:
        for line in file:
            partes = line.strip().split()
            if len(partes) != 4:  # Asegurarse de que hay 4 partes en cada línea
                continue  # Ignorar líneas que no siguen el formato esperado
            numero_documento = partes[1][:-4]  # Obtener el número de documento eliminando la extensión .txt
            try:
                similitud = float(partes[-1])  # Convertir la similitud a un número decimal
            except ValueError:
                continue  # Ignorar líneas donde la similitud no es un número válido
            documentos_relevantes[numero_documento] = similitud
    return documentos_relevantes


In [20]:
# Leer documentos relevantes
documentos_relevantes = leer_documentos_relevantes(r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\catslimpia.txt')
#documentos_relevantes = leer_documentos_relevantes('C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\catslimpia.txt.txt')


In [21]:
def calcular_precision_recall(resultados_recuperados, resultados_relevantes):
    num_resultados_recuperados = len(resultados_recuperados)
    num_resultados_relevantes = len(resultados_relevantes)
    
    resultados_comunes = set(resultados_recuperados) & set(resultados_relevantes)
    num_resultados_comunes = len(resultados_comunes)
    
    precision = num_resultados_comunes / num_resultados_recuperados if num_resultados_recuperados > 0 else 0
    recall = num_resultados_comunes / num_resultados_relevantes if num_resultados_relevantes > 0 else 0
    
    return precision, recall

In [22]:
def leer_consultas(archivo):
    consultas = []
    with open(archivo, 'r') as file:
        for line in file:
            consulta = line.strip()
            consultas.append(consulta)
    return consultas

In [23]:
# Leer consultas
ruta_archivo_consultas = r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\querys.txt'
consultas = leer_consultas(ruta_archivo_consultas)

In [24]:
resultados_totales = {}  # Almacenar los resultados de todas las consultas
for consulta in consultas:
    resultados_recuperados = engine.search(consulta)
    resultados_totales[consulta] = resultados_recuperados

In [28]:
# Calcular precisión y recall para cada consulta
precisiones = []
recalls = []
for consulta, documentos_recuperados in resultados_totales.items():
    precision, recall = calcular_precision_recall(documentos_recuperados, documentos_relevantes)
    precisiones.append(precision)
    recalls.append(recall)

# Metricas de Evaluacion

In [29]:
# Calcular la precisión y el recall promedio para todas las consultas
precision_promedio = sum(precisiones) / len(precisiones)
recall_promedio = sum(recalls) / len(recalls)

print("Precisión promedio:", precision_promedio)
print("Recall promedio:", recall_promedio)

Precisión promedio: 0.0
Recall promedio: 0.0
